<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/llm/rinna3_6b_instruction_ppo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation

In [ ]:
!pip install transformers accelerate bitsandbytes sentencepiece

Load model

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "rinna/japanese-gpt-neox-3.6b-instruction-ppo"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map={"": 0},
    # load_in_4bit=True,
    load_in_8bit=True,
    # torch_dtype=torch.float16,
)

In [2]:
last_prompt = ""
while True:

    user_input = input("ユーザー: ")
    if user_input == "":
        break

    prompt = last_prompt + "<NL>ユーザー: " + user_input + "<NL>システム: "
    token_ids = tokenizer.encode(
        prompt, add_special_tokens=False, return_tensors="pt")

    with torch.no_grad():
        output_ids = model.generate(
            token_ids.to(model.device),
            do_sample=True,
            max_new_tokens=128,
            temperature=0.7,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):])

    last_prompt = prompt + "システム: " + output
    output = output.replace("<NL>", "\n")
    output = "りんな: " + output.replace("</s>", "")
    print(output)

ユーザー: 日本で
りんな: 日本政府は、20年のオリンピック・パラリンピック競技大会を成功裏に開催するために重要な役割を果たしました。
ユーザー: 重要な役割について教えて
りんな: もちろんです。オリンピック・パラリンピック競技大会は、世界の国々が協力して世界的なスポーツイベントを開催する重要な機会です。日本オリンピック委員会や国内競技連盟など、数多くの組織によって運営されています。これらの組織には、オリンピック・パラリンピック競技大会に関連するさまざまな法律や規制があります。オリンピック・パラリンピック競技大会は、世界から訪れるアスリートや観客にとって非常に重要で印象的です。
ユーザー: どんな選手が出たの？
りんな: まず、アメリカのホッケーリーグでプレーする最も注目すべき選手を紹介します。1人目は、アメリカのホッケーリーグを代表するフォワードであるアイバン・モレスモです。彼は、アメリカのホッケーリーグで歴代3位の得点数を誇る優れたプレイヤーであり、ホッケーの殿堂入りも果たしています。また、彼はホッケーのアメリカ代表としても非常に重要な存在です。2人目は、カナダのホッケーリーグでプレーするセンターのダニエル・ボーダーマンです。ボーダーマンは、ホッケーのカナダ代表としても非常に重要な存在です。3人目は、スイスのホッケーリーグでプレーするゴールキーパーのマルコス・ボネーラです。ボネーラは、ホッケーのスイス
ユーザー: サッカーでは
りんな: サッカーでも、ホッケーと同様に優れたフォワードやミッドフィールダーが活躍しています。最も有名な選手には、ドイツのブンデスリーガでプレーするマリオ・ゲッツェと、イタリアのセリエAでプレーする長友佑都がいます。
ユーザー: 
